In [1]:
# (hypothetical) state populations
p = [27744, 25178, 19951, 14610, 9225, 3292]

# number of states
n = len(p)

# total country population
p_total = sum( p[i] for i in range(n) )

# total number of seats to distribute
k = 36

# state quotas
q = [ k * p[i] / p_total for i in range(n) ]

print("Populations:",p)
print("Quotas:",q)

Populations: [27744, 25178, 19951, 14610, 9225, 3292]
Quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# create model object
m = gp.Model()

# create integer vars x, with x[i] being the number of seats for state i
x = m.addVars( n, vtype=GRB.INTEGER )

# distribute k seats
m.addConstr( gp.quicksum( x[i] for i in range(n) ) == k )

# each state gets at least one seat
for i in range(n):
    x[i].LB = 1

m.update()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-06-27
Using license file C:\Users\buchanan\gurobi.lic


In [4]:
# Add objective function for Huntington-Hill's method: 
#     min sum_i x_i * ( p_i / x_i - d )^2, where
d = p_total / k

# Expanding gives the objective
#     min sum_i [ (p_i)^2 / x_i - 2*d*p_i/x_i + d^2*x_i ].

# Substituting y_i = 1 / x_i gives
#     min sum_i [ (p_i)^2*y_i - 2*d*p_i*y_i + d^2*x_i ].

y = m.addVars(n)
m.addConstrs( x[i] * y[i] == 1 for i in range(n) )

m.setObjective( gp.quicksum( p[i]*p[i]*y[i] - 2*d*p[i]*y[i] - d*d*x[i] for i in range(n) ), GRB.MINIMIZE )

m.Params.NonConvex = 2
m.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 12 columns and 6 nonzeros
Model fingerprint: 0xc26326f0
Model has 6 quadratic constraints
Variable types: 6 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e+06, 6e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+01]
  QRHS range       [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 25 rows, 12 columns, 54 nonzeros
Presolved model has 6 bilinear constraint(s)
Variable types: 6 continuous, 6 integer (0 binary)

Root relaxation: objective -2.037572e+08, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [5]:
sol = [ x[i].x for i in range(n) ]
print("Optimal solution :", sol)
print("Compare to quotas:", q)

Optimal solution : [11.0, 9.000000000000002, 6.999999999999998, 5.0, 3.0, 1.0]
Compare to quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]
